# Image Generator Backend Deployment Notebook

In [1]:
import base64
import io
import os

import covalent_cloud as cc
import torch
from covalent_cloud.cloud_executor import GPU_TYPE
from diffusers import AutoPipelineForText2Image

cc.save_api_key(os.environ["CC_API_KEY"])

In [2]:
ENV_NAME = "sdxl-demo-backend"

cc.create_env(
    name=ENV_NAME,
    pip=[
        "accelerate==0.29.1",
        "diffusers==0.27.2",
        "sentencepiece==0.2.0",
        "torch==2.2.2",
        "transformers==4.39.3",
    ],
    wait=True,
)

Environment Already Exists.


In [3]:
gpu_executor = cc.CloudExecutor(
    env=ENV_NAME,
    num_cpus=25,
    memory="56 GB",
    time_limit="15 days",
    num_gpus=1,
    gpu_type=GPU_TYPE.L40,
)

@cc.service(executor=gpu_executor, name="SDXL Image Generator Service")
def text_to_image_service(model_path: str):
    """Creates an SDXL Image Generator service"""
    pipeline = AutoPipelineForText2Image.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        variant="fp16",
    ).to("cuda")

    return {"pipeline": pipeline}

In [4]:
@text_to_image_service.endpoint(route="/text-to-image")
def generate_image(pipeline, prompt: str, num_inference_steps: int = 1):
    """Generate an image based on a prompt"""
    image = pipeline(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=0.0).images[0]

    bytes_io = io.BytesIO()
    image.save(bytes_io, format='PNG')
    image_as_str = base64.b64encode(bytes_io.getvalue()).decode('utf-8')
    return image_as_str

#### When ready, copy backend base URL (address) into streamlit app

In [5]:
info = cc.deploy(text_to_image_service)(model_path="stabilityai/sdxl-turbo")
info = cc.get_deployment(info.function_id, wait=True)
print(info)
print(info.address)

╭──────────────────────── Deployment Information ────────────────────────╮
│  Name          SDXL Image Generator Service                            │
│  Description   Creates an SDXL Image Generator service                 │
│  Function ID   66563437f7d37dbf2a468c9f                                │
│  Address       https://fn.prod.covalent.xyz/166563437f7d37dbf2a468c9f  │
│  Status        ACTIVE                                                  │
│  Tags                                                                  │
│  Auth Enabled  Yes                                                     │
╰────────────────────────────────────────────────────────────────────────╯
╭────────────────────────────────────────────────────╮
│                POST /text-to-image                 │
│  Streaming    No                                   │
│  Description  Generate an image based on a prompt  │
╰────────────────────────────────────────────────────╯
Authorization token: <redacted-authorization-token

#### Danger Zone!

In [ ]:
# info.teardown()